In [1]:
import cv2
from inference import infer_image, load_models
import numpy as np
# Load models
deepc_path = './reference/longrun-epoch=99-step=369700.ckpt'
refinenet_path = '/home/alessia/deepcharuco/src_100epochs/tb_logs/ckpts_refinenet/epoch=99-step=34400.ckpt'
n_ids = 9
 # The number of corners (models pretrained use 16 for default board)
device = 'cuda'   # use device: cpu / mps / cuda
refinenet = load_models(refinenet_path, device='cpu')

# Run inference on BGR image
img = cv2.imread('/home/alessia/Desktop/training_dataset/render_00111.png', cv2.IMREAD_GRAYSCALE)


/home/alessia/mambaforge/envs/deepcharuco/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

cv2.namedWindow("image", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("image", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
print(img.shape)
# cv2.imshow('image',img)
# cv2.waitKey(0)
cv2.destroyAllWindows()

(2176, 4112)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [3]:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

In [4]:
#gray = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
detected_corners, ids, rejected = cv2.aruco.detectMarkers(img, dictionary, parameters=parameters)
print(detected_corners)
TERMINATION_CRITERIA = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

corners_refined = cv2.cornerSubPix(img, detected_corners[0][0], (11, 11), (-1, -1), TERMINATION_CRITERIA)
print(corners_refined)

(array([[[1001.,  181.],
        [ 999.,  282.],
        [ 896.,  258.],
        [ 895.,  153.]]], dtype=float32),)
[[1000.5511   179.51129]
 [ 999.8262   282.53683]
 [ 895.97797  260.9258 ]
 [ 893.87384  153.51767]]


In [5]:
img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
# corners_refined=np.array([[1486.3987 ,  449.76825],
#             [1493.8314  , 532.8715 ],
#             [1405.0139  , 540.29785],
#             [1397.0697 ,  449.87808]])

In [6]:

keypoints, out_img = infer_image(img, corners_refined,4, refinenet, draw_pred=True, device='cpu')

print(keypoints)


[[9.98926086e+02 1.75761292e+02 0.00000000e+00]
 [1.00282617e+03 2.83661835e+02 1.00000000e+00]
 [8.93102966e+02 2.60800812e+02 2.00000000e+00]
 [8.96998840e+02 1.52142670e+02 3.00000000e+00]]


/home/alessia/deepcharuco/src_100epochs/./models/model_utils.py:42: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  row_indices = indices // x.shape[2]


In [15]:
cv2.namedWindow('out img', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('out img', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
cv2.imshow('out img',out_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:

# # The out_img will have corners plotted on it if draw_pred is True
# # The keypoints format is (x, y, id_corner)
# keypoints, out_img = infer_image(img, n_ids, deepc, refinenet, draw_pred=True, device=device)
from inference import solve_pnp

# load your intrinsics as camera_matrix and dist_coeffs
# check calib_intrinsics for a reference how I generated them.
# Check the solve pnp function and adapt it to your needs.
# ret, rvec, tvec = solve_pnp(keypoints, col_count, row_count, square_len,
#                             camera_matrix, dist_coeffs)